# Tests

In [1]:
import os
import sys
import logging
import requests
import json
import re
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips
from lib.gapp import BSSH
from lib.samplesheet import SampleSheet

nq    = Nanuq()
pho   = Phenotips()
bssh  = BSSH()

class Args:
    # Simulate an argparse instance
    #
    def __init__(self, run='LH00336_0006', level='info'):
        self.run   = run
        self.level = level

args = Args(level='debug')
vars(args)

{'run': 'LH00336_0006', 'level': 'debug'}

In [2]:
sheet = SampleSheet('D:\\HSJ\\Projects\\PRAGMatIQ\\Runs\\Seq_10B_PRAG_20240130.csv')
sheet.sections['DragenGermline_Data'][0]

['Sample_ID',
 'ReferenceGenomeDir',
 'VariantCallingMode',
 'QcCoverage1BedFile',
 'QcCoverage2BedFile',
 'QcCoverage3BedFile',
 'QcCrossContaminationVcfFile']

In [20]:
def list_dragengermline_samples(samplesheet):
    samples = []
    with open(samplesheet, 'r') as fh:
        for line in fh.readlines():
            if line.startswith('['):
                section = line
            else:
                if section.startswith('[DragenGermline_Data]'):
                    cols = line.split(',')
                    if not line.startswith('Sample_ID') and len(cols) > 1:
                        samples.append(cols[0])
    return samples

for sample in list_dragengermline_samples('D:\\HSJ\\Projects\\PRAGMatIQ\\Runs\\Seq_10B_PRAG_20240130.csv'):
    print(sample)


24159
24160
24161
24162
24163
24164
24212
24213
24214
24215
24216
24217
24251
24252
24253
24254
24255
24256


In [ ]:
# Get authorization token from Emedgene
# Please note - the Authorization header is only valid for a limited time, and
# expires after 8H. In that case, any request made with an expired token will 
# return a 403 code. To resolve, re-do the Login procedure to get a new token.
#
url      = "https://chusaintejustine.emedgene.com/api/auth/api_login/"
payload  = '{"username": "cqgc.bioinfo.hsj@ssss.gouv.qc.ca", "password": "3175CoteSainte-Catherine"}'
headers  = {'Content-Type': 'application/json'}
response = requests.request("POST", url, headers=headers, data=payload)
auth_header = response.json()["Authorization"]
auth_header


In [ ]:
# Get case for EMG634551172 (GM240123)
#
case_id = "EMG634551172"
get_test_response = requests.get(f'https://chusaintejustine.emedgene.com/api/test/{case_id}/', json={}, headers={'Authorization': auth_header})
# with open('D:\\HSJ\\Projects\\PRAGMatIQ\\emg_scripts\\emg_case_example-GM240123.json', 'w') as fh:
#     json.dump(get_test_response.json(), fh, indent=4)
get_test_response.json().keys()


In [ ]:
# Transfer files to aws
# --profile "emedgene" or "emedgene-eval"
aws s3 ls --profile emedgene-eval s3://cac1-prodca-emg-downloads/Ste_Justine_eval/upload